In [92]:
import pandas as pd
import pickle
from string import ascii_uppercase as alphabet
import json

In [77]:
all_tables = pd.read_html("https://en.wikipedia.org/wiki/UEFA_Euro_2024")

In [85]:
dict_table = {}

for letter, i in zip(alphabet, range(18, 57, 7)):
    print(f"Group {letter}")
    df = all_tables[i]
    df = df.rename(columns={df.columns[1]:'Team'})
    df = df.drop("Qualification", axis=1)
    dict_table[f"Group {letter}"] = df

Group A
Group B
Group C
Group D
Group E
Group F


In [86]:
dict_table["Group A"]

,Pos,Team,Pld,W,D,L,GF,GA,GD,Pts
0,1,Germany (H),0,0,0,0,0,0,0,0
1,2,Scotland,0,0,0,0,0,0,0,0
2,3,Hungary,0,0,0,0,0,0,0,0
3,4,Switzerland,0,0,0,0,0,0,0,0


In [289]:
with open ("_data/dict_table", "wb") as output:
    pickle.dump(dict_table, output)

In [112]:
from bs4 import BeautifulSoup
import requests

# Generate list of years from 1960 to 2020 in 4-year increments
years = list(range(1960, 2021, 4))

def get_matches(year):
    web = f'https://en.wikipedia.org/wiki/UEFA_Euro_{year}'
    response = requests.get(web)
    content = response.text
    soup = BeautifulSoup(content, "lxml")
    matches = soup.find_all("div", class_="footballbox")

    home = []
    score = []
    away = []

    for match in matches:
        home.append(match.find('th', class_='fhome').get_text())
        score.append(match.find('th', class_='fscore').get_text())
        away.append(match.find('th', class_='faway').get_text())

    dict_football = {"home": home, "score": score, "away": away}
    df_football = pd.DataFrame(dict_football)
    df_football["year"] = year
    return df_football


In [142]:
# Extracting historic Euros scores
Euros = [get_matches(year) for year in years]
df_Euros = pd.concat(Euros, ignore_index=True)
df_Euros.to_csv("_data/Euros_historical_data.csv", index=False)

In [143]:
# Extracting this years fixtures
df_fixture = get_matches(2024)
df_fixture["Match_no"] = df_fixture['score'].str.extract('(\d+)').astype(int)
df_fixture.sort_values("Match_no").to_csv("_data/Euros_fixtures.csv", index=False)

In [200]:
df_historical = pd.read_csv("_data/Euros_historical_data.csv")
df_fixture = pd.read_csv("_data/Euros_fixtures.csv")

In [201]:
df_fixture["home"] = df_fixture["home"].str.strip()
df_fixture["away"] = df_fixture["away"].str.strip()

In [202]:
df_historical_data = (
    df_historical
    .drop_duplicates()
    .sort_values("year")
)

In [203]:
# Clean score, home/away cols
df_historical_data["score"] = df_historical_data["score"].str.replace("[^\d\–]","", regex=True)
df_historical_data["home"] = df_historical_data["home"].str.strip()
df_historical_data["away"] = df_historical_data["away"].str.strip()

# Splitting score into home & away cols
df_historical_data[["HomeGoals", "AwayGoals"]] = df_historical_data["score"].str.split("–", expand=True)
df_historical_data.drop("score", axis=1, inplace=True)

# Renaming cols and changing data types
df_historical_data.rename(columns={"home": "HomeTeam", "away": "AwayTeam",
                                   "year": "Year"}, inplace=True)
df_historical_data = df_historical_data.astype({"HomeGoals": int, "AwayGoals": int,
                                                "Year": int})

# Creating new col "totalgoals"
df_historical_data["TotalGoals"] = df_historical_data["HomeGoals"] + df_historical_data["AwayGoals"]

In [204]:
df_historical_data.to_csv("_data/clean_Euros_matches.csv", index=False)
df_fixture.to_csv("_data/clean_Euros_fixtures.csv", index=False)

In [292]:
dict_table = pickle.load(open("_data/dict_table", "rb"))
df_historical_clean = pd.read_csv("_data/clean_Euros_matches.csv")
df_fixture_clean = pd.read_csv("_data/clean_Euros_fixtures.csv")

In [293]:
# Replace "Germany" with "Germany (H)" in both cols
df_updated = df_historical_data.copy() # Copy to avoid overwriting original df
df_updated.loc[df_updated["HomeTeam"] == "Germany", "HomeTeam"] = "Germany (H)"
df_updated.loc[df_updated["AwayTeam"] == "Germany", "AwayTeam"] = "Germany (H)"

df_historical_clean = df_updated

In [294]:
df_home = df_historical_clean[["HomeTeam", "HomeGoals", "AwayGoals"]]
df_away = df_historical_clean[["AwayTeam", "HomeGoals", "AwayGoals"]]

df_home = df_home.rename(columns={"HomeTeam":"Team", "HomeGoals":"GoalsScored",
                                  "AwayGoals":"GoalsConceded"})
df_away = df_away.rename(columns={"AwayTeam":"Team", "HomeGoals":"GoalsConceded",
                                  "AwayGoals":"GoalsScored"})

df_team_strength = pd.concat([df_home, df_away], ignore_index=True).groupby(["Team"]).mean()
df_team_strength

,GoalsScored,GoalsConceded
Team,,
Albania,0.333333,1.000000
Austria,0.700000,1.200000
Belgium,1.409091,1.272727
Bulgaria,0.666667,2.166667
CIS,0.333333,1.333333
Croatia,1.363636,1.272727
Czech Republic,1.241379,1.275862
Czechoslovakia,1.500000,1.250000
Denmark,1.272727,1.515152


In [295]:
# Replace with "Germany (H)"
df_fixture.replace({"home": "Germany"}, {"home": "Germany (H)"}, inplace=True)
df_fixture.replace({"away": "Germany"}, {"away": "Germany (H)"}, inplace=True)

,home,score,away,year,Match_no
0,Germany (H),Match 1,Scotland,2024,1
1,Hungary,Match 2,Switzerland,2024,2
2,Spain,Match 3,Croatia,2024,3
3,Italy,Match 4,Albania,2024,4
4,Serbia,Match 5,England,2024,5
5,Slovenia,Match 6,Denmark,2024,6
6,Poland,Match 7,Netherlands,2024,7
7,Austria,Match 8,France,2024,8
8,Belgium,Match 9,Slovakia,2024,9
9,Romania,Match 10,Ukraine,2024,10


In [296]:
from scipy.stats import poisson

# Function to predict the points gained from a match for each team
def predict_points(home, away):
    if home in df_team_strength.index and away in df_team_strength.index:
        # goals_scored * goals_conceded
        lamb_home = df_team_strength.at[home, 'GoalsScored'] * df_team_strength.at[away, "GoalsConceded"]
        lamb_away = df_team_strength.at[away, 'GoalsScored'] * df_team_strength.at[home, "GoalsConceded"]

        prob_home, prob_away, prob_draw = 0, 0, 0

        for x in range(0, 11):
            for y in range(0, 11):
                p = poisson.pmf(x, lamb_home) * poisson.pmf(y, lamb_away)

                if x == y:
                    prob_draw += p
                elif x > y:
                    prob_home += p
                elif x < y:
                    prob_away += p
    
        points_home = 3 * prob_home + prob_draw
        points_away = 3 * prob_away + prob_draw

        return(points_home, points_away)
    
    else:
        return(0,0)

In [297]:
print(predict_points("Germany (H)", "Scotland"))

(2.0562714008680647, 0.6905725684869202)


In [298]:
# Setting out the fixture rounds
df_fixture_group44 = df_fixture[:36].copy()
df_fixture_R16 = df_fixture[36:44].copy()
df_fixture_quarter = df_fixture[44:48].copy()
df_fixture_semi = df_fixture[48:50].copy()
df_fixture_final = df_fixture[50:].copy()

In [299]:
for group in dict_table:
    teams_in_group = dict_table[group]["Team"].values
    df_fixture_group6 = df_fixture_group44[df_fixture_group44["home"].isin(teams_in_group)]

    for index, row in  df_fixture_group6.iterrows():
        home, away = row["home"], row["away"]
        points_home, points_away = predict_points(home, away)
        dict_table[group].loc[dict_table[group]["Team"] == home, "Pts"] += int(points_home)
        dict_table[group].loc[dict_table[group]["Team"] == away, "Pts"] += int(points_away)

    dict_table[group] = dict_table[group].sort_values("Pts", ascending = False).reset_index()
    dict_table[group] = dict_table[group][["Team", "Pts"]]
    dict_table[group] = dict_table[group].round(0)


In [307]:
dict_table["Group B"].iloc[2]["Pts"]

2.0

In [391]:
import pandas as pd

# Create an empty DataFrame for ranking of 3rd place teams
third_placed_teams = pd.DataFrame(columns=["Pos", "Grp", "Team", "Pld",
                                           "W", "D", "L", "GF", "GA", "GD", "Pts", "Qualification"]).reset_index(drop=True)

# Iterate over each group in dict_table
for i, group in enumerate(dict_table, start=1):
    # Calculate points for teams in the group based on predictions
    teams_in_group = dict_table[group]["Team"].values

    for index, row in df_fixture_group6.iterrows():
        home, away = row["home"], row["away"]
        points_home, points_away = predict_points(home, away)
        dict_table[group].loc[dict_table[group]["Team"] == home, "Pts"] += points_home
        dict_table[group].loc[dict_table[group]["Team"] == away, "Pts"] += points_away

    # Sort teams in the group based on points
    dict_table[group] = dict_table[group].sort_values("Pts", ascending=False).reset_index(drop=True)
    dict_table[group] = dict_table[group][["Team", "Pts"]]
    dict_table[group] = dict_table[group].round(0)

    # Add third-placed team to the ranking DataFrame
    third_place_team = dict_table[group].iloc[2]
    add_third_place = pd.DataFrame([{
        "Pos": i,
        "Grp": group,
        "Team": third_place_team["Team"],
        "Pld": 0,
        "W": 0,
        "D": 0,
        "L": 0,
        "GF": 0,
        "GA": 0,
        "GD": 0,
        "Pts": third_place_team["Pts"],
        "Qualification": ""
    }])

    # Concatenate the new row to the existing DataFrame
    third_placed_teams = pd.concat([third_placed_teams.astype(add_third_place.dtypes),
                                    add_third_place.astype(third_placed_teams.dtypes)], axis=0, ignore_index=True)

# Sort third placed teams based on points
third_placed_teams = third_placed_teams.sort_values(by="Pts", ascending=False).reset_index(drop=True)

# Determine which teams advance to knockout phase (top 4)
top_four_third_teams = third_placed_teams.head(4)
top_four_third_teams.loc[:, "Qualification"] = "Advance to knockout phase"

# Display the filled table
top_four_third_teams

,Pos,Grp,Team,Pld,W,D,L,GF,GA,GD,Pts,Qualification
0,6,Group F,Turkey,0,0,0,0,0,0,0,25.0,Advance to knockout phase
1,1,Group A,Scotland,0,0,0,0,0,0,0,3.0,Advance to knockout phase
2,5,Group E,Slovakia,0,0,0,0,0,0,0,3.0,Advance to knockout phase
3,2,Group B,Croatia,0,0,0,0,0,0,0,2.0,Advance to knockout phase


In [392]:
df_fixture_final

,home,score,away,year,Match_no
50,Winner Match 49,Match 51,Winner Match 50,2024,51


In [402]:
for group in dict_table:
    group_winner = dict_table[group].loc[0, "Team"]
    runners_up = dict_table[group].loc[1, "Team"]
    third_place = dict_table[group].loc[2, "Team"]
    df_fixture_R16.replace({f"Winner {group}": group_winner,
                            f"Runner-up {group}": runners_up,
                            f"3rd {group}": third_place}, inplace=True)

df_fixture_R16["winner"] = "?"
df_fixture_R16

,home,score,away,year,Match_no,Winner,winner
36,Germany (H),Match 37,Denmark,2024,37,?,?
37,Hungary,Match 38,Italy,2024,38,?,?
38,Spain,Match 39,Poland,2024,39,?,?
39,England,Match 40,Slovakia,2024,40,?,?
40,Portugal,Match 41,Croatia,2024,41,?,?
41,France,Match 42,Romania,2024,42,?,?
42,Belgium,Match 43,Scotland,2024,43,?,?
43,Netherlands,Match 44,Czech Republic,2024,44,?,?


In [403]:
third_place = dict_table["Group A"].loc[2, "Team"]
third_place

'Scotland'

In [404]:
# Replace "3rd Group D/E/F" with third-place team from Group E
df_fixture_R16["away"].replace("3rd Group D/E/F", dict_table["Group E"].loc[2, "Team"], inplace=True)

# Replace "3rd Group A/D/E/F" with third-place team from Group D
df_fixture_R16["away"].replace("3rd Group A/D/E/F", dict_table["Group D"].loc[2, "Team"], inplace=True)

# Replace "3rd Group A/B/C" with third-place team from Group B
df_fixture_R16["away"].replace("3rd Group A/B/C", dict_table["Group B"].loc[2, "Team"], inplace=True)

# Replace "3rd Group A/B/C/D" with third-place team from Group A
df_fixture_R16["away"].replace("3rd Group A/B/C/D", dict_table["Group A"].loc[2, "Team"], inplace=True)

C:\Users\jtufts\AppData\Local\Temp\ipykernel_20272\2797546556.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_fixture_R16["away"].replace("3rd Group D/E/F", dict_table["Group E"].loc[2, "Team"], inplace=True)
C:\Users\jtufts\AppData\Local\Temp\ipykernel_20272\2797546556.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which w

In [405]:
df_fixture_R16

,home,score,away,year,Match_no,Winner,winner
36,Germany (H),Match 37,Denmark,2024,37,?,?
37,Hungary,Match 38,Italy,2024,38,?,?
38,Spain,Match 39,Poland,2024,39,?,?
39,England,Match 40,Slovakia,2024,40,?,?
40,Portugal,Match 41,Croatia,2024,41,?,?
41,France,Match 42,Romania,2024,42,?,?
42,Belgium,Match 43,Scotland,2024,43,?,?
43,Netherlands,Match 44,Czech Republic,2024,44,?,?


In [406]:
def get_winner(df_fixture_updated):
    for index, row in df_fixture_updated.iterrows():
        home, away = row["home"], row["away"]
        points_home, points_away = predict_points(home, away)
        
        if points_home > points_away:
            winner = home
        else:
            winner = away

        df_fixture_updated.loc[index, "winner"] = winner
    return df_fixture_updated

In [407]:
get_winner(df_fixture_R16)

df_fixture_R16

,home,score,away,year,Match_no,Winner,winner
36,Germany (H),Match 37,Denmark,2024,37,?,Germany (H)
37,Hungary,Match 38,Italy,2024,38,?,Italy
38,Spain,Match 39,Poland,2024,39,?,Spain
39,England,Match 40,Slovakia,2024,40,?,England
40,Portugal,Match 41,Croatia,2024,41,?,Portugal
41,France,Match 42,Romania,2024,42,?,France
42,Belgium,Match 43,Scotland,2024,43,?,Belgium
43,Netherlands,Match 44,Czech Republic,2024,44,?,Netherlands


In [408]:
def update_table(df_fixture_round_1, df_fixture_round_2):
    for index, row in df_fixture_round_1.iterrows():
        winner = df_fixture_round_1.loc[index, "winner"]
        match = df_fixture_round_1.loc[index, "score"]
        df_fixture_round_2.replace({f"Winner {match}": winner}, inplace=True)
    
    df_fixture_round_2["Winner"] = "?"
    return df_fixture_round_2

In [410]:
update_table(df_fixture_R16, df_fixture_quarter)

,home,score,away,year,Match_no,Winner
44,Spain,Match 45,Germany (H),2024,45,?
45,Portugal,Match 46,France,2024,46,?
46,Belgium,Match 47,Netherlands,2024,47,?
47,England,Match 48,Italy,2024,48,?


In [411]:
get_winner(df_fixture_quarter)

,home,score,away,year,Match_no,Winner,winner
44,Spain,Match 45,Germany (H),2024,45,?,Spain
45,Portugal,Match 46,France,2024,46,?,Portugal
46,Belgium,Match 47,Netherlands,2024,47,?,Netherlands
47,England,Match 48,Italy,2024,48,?,Italy


In [412]:
update_table(df_fixture_quarter, df_fixture_semi)

,home,score,away,year,Match_no,Winner
48,Spain,Match 49,Portugal,2024,49,?
49,Netherlands,Match 50,Italy,2024,50,?


In [413]:
get_winner(df_fixture_semi)

,home,score,away,year,Match_no,Winner,winner
48,Spain,Match 49,Portugal,2024,49,?,Spain
49,Netherlands,Match 50,Italy,2024,50,?,Italy


In [415]:
update_table(df_fixture_semi, df_fixture_final)

,home,score,away,year,Match_no,Winner
50,Spain,Match 51,Italy,2024,51,?


In [416]:
get_winner(df_fixture_final)

,home,score,away,year,Match_no,Winner,winner
50,Spain,Match 51,Italy,2024,51,?,Italy
